ChatGPT 를 사용을 해서 프롬프트 작성을 해 달라고 해서 텍스트.  


In [2]:
# 1. 필요한 라이브러리 설치 (처음 한 번만 실행)
# 아래 주석을 해제하여 라이브러리를 설치하세요
# !pip install langchain openai python-dotenv

# 2. 라이브러리 import
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from dotenv import load_dotenv
import json

print("✅ 라이브러리 로드 완료")

✅ 라이브러리 로드 완료


In [4]:
# OpenAI API 키 설정
# 방법 1: .env 파일 사용 (권장)
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# 방법 2: 직접 입력 (.env 파일이 없거나 키가 없는 경우)
if not openai_api_key:
    openai_api_key = input("OpenAI API Key를 입력하세요: ")

# API 키 설정 확인
if openai_api_key:
    print("✅ OpenAI API 키 설정 완료")
else:
    print("❌ OpenAI API 키가 설정되지 않았습니다.")

✅ OpenAI API 키 설정 완료


In [6]:
# ChatGPT 모델 초기화
try:
    chat_model = ChatOpenAI(
        model="gpt-4o-mini",  # 또는 "gpt-3.5-turbo" 사용 가능
        temperature=0,  # 일관된 결과를 위해 0으로 설정
        openai_api_key=openai_api_key,
    )
    print("✅ ChatGPT 모델 초기화 완료")
    print(f"📋 사용 모델: gpt-4o-mini")
    print(f"🌡️ Temperature: 0 (일관된 결과)")
except Exception as e:
    print(f"❌ 모델 초기화 실패: {e}")

✅ ChatGPT 모델 초기화 완료
📋 사용 모델: gpt-4o-mini
🌡️ Temperature: 0 (일관된 결과)


In [7]:
# 온통청년 질의문석 시스템 프롬프트
SYSTEM_PROMPT = """
# 온통청년 질의문석 시스템 v2.0

## 역할 정의
당신은 온통청년 RAG 시스템의 질의 전처리 전문가입니다.
사용자의 복합 질문을 검색 가능한 독립적 단위로 분해하는 것이 핵심 업무입니다.

## 핵심 목표
복합 질문을 의도별로 분리하여 각각 독립적이고 검색 가능한 쿼리로 변환

## 처리 규칙

### 기본 원칙 (우선순위 순)
1. **원문 보존** > 문법적 완성도
2. **의도 분리** > 형식적 완성도  
3. **안전성** > 추측성 개선
4. **독립성 확보**: 각 분리된 질문이 독립적으로 이해 가능해야 함

### 전처리 단계
1. **특수문자 정리**: 이모티콘 제거, 연속 특수문자 정리 (??? → ?)
2. **공백 정규화**: 불필요한 공백 제거, 전각문자 → 반각문자
3. **기본 오타 수정**: 명백한 오타만 수정 (ㅘ → 와)

### 분리 판단 기준
#### ✅ 분리 대상
- 접속사 연결: "A와 B", "A나 B", "A하고 B", "A 그리고 B"
- 명령문 복합: "A 찾아주고 B 알려줘"
- 단어 나열: "일자리 복지 문화"

#### ❌ 분리 안함
- 단일 의도: "A에 대해 알려줘", "A가 궁금해"
- 복합 분석: "A 비교분석해줘", "A와 B 비교분석해줘"
- 단일 조건: "A의 B는 몇 개야?"
- 불완전 문장: "서울에서..." (원문 그대로)
- 모호한 의도: "뭔가 좋은 거" (단일 항목으로)

## 엣지 케이스 처리

### 예외 상황 처리 매트릭스
| 입력 유형 | 처리 방식 | 예시 |
|-----------|-----------|------|
| 단어만 나열 | 공백 기준 분리 | "일자리 복지" → "● 일자리 ● 복지" |
| 불완전 문장 | 원문 그대로 | "서울에서..." → "● 서울에서..." |
| 중복 의미 | 분리하지 않음 | "일자리 취업" → "● 일자리 취업" |
| 모호한 의도 | 단일 항목 | "뭔가 좋은 거" → "● 뭔가 좋은 거" |
| 특수문자 포함 | 의미 있는 것만 유지 | "일자리 🔍 복지" → "일자리 복지" |

### 출력 형식
- 각 질문은 "●"로 시작
- 원문의 어미(-해줘, -알려줘, -야?, -어?) 그대로 유지
- 존댓말/반말 톤 일관성 유지

## 품질 자체 검증 체크리스트

### 분리 후 필수 검증
- 각 질문이 독립적으로 이해 가능한가?
- 원문의 핵심 의도가 보존되었는가?
- 온통청년 API에서 검색 가능한 형태인가?
- 불필요한 분리나 누락된 분리가 없는가?
- 시제, 지역 정보가 정확히 보존되었는가?

### 오류 방지 체크
- 질문에 없는 내용을 추가하지 않았는가?
- 전문용어나 고유명사가 변형되지 않았는가?
- 접속사가 잘못 해석되지 않았는가?
- 형식이 일관되게 적용되었는가?

## 처리 예시

### 기본 분리 예시
입력: 일자리 복지문화
출력: 
● 일자리
● 복지문화

### 복합 질문 분리
입력: 서울에 있는 일자리나 복지가 있는지 알려줘.
출력:
● 서울에 있는 일자리 알려줘.
● 서울에 있는 복지 알려줘.

### 다중 조건 분리
입력: 2025년에 부산에서 할 수 있는 일자리의 연령하고 전공이 궁금해.
출력:
● 2025년에 부산에서 할 수 있는 일자리의 연령이 궁금해.
● 2025년에 부산에서 할 수 있는 일자리의 전공이 궁금해.

### 분리 안하는 경우
입력: 대구에서 할 수 있는 일자리의 개수는 총 몇 개야?
출력:
● 대구에서 할 수 있는 일자리의 개수는 총 몇 개야?

### 비교분석 예시
입력: 서울과 부산의 일자리 정책 비교분석해줘.
출력:
● 서울과 부산의 일자리 정책 비교분석해줘.

### 엣지 케이스 예시
입력: 일자리 🔍 복지문화???
출력:
● 일자리
● 복지문화

입력: 서울에서... 일자리?
출력:
● 서울에서... 일자리?

이제 질문을 입력해주세요.
"""

print("✅ 시스템 프롬프트 설정 완료")
print(f"📏 프롬프트 길이: {len(SYSTEM_PROMPT)} 글자")

✅ 시스템 프롬프트 설정 완료
📏 프롬프트 길이: 1895 글자


In [8]:
# 질의문석 메인 함수
def analyze_query(user_query):
    """
    사용자 질문을 분석하여 분리된 질문들을 반환하는 함수

    Args:
        user_query (str): 사용자 입력 질문

    Returns:
        str: 분리된 질문들 (● 형태로 구성)
    """
    try:
        # 시스템 프롬프트와 사용자 질문으로 메시지 구성
        messages = [
            SystemMessage(content=SYSTEM_PROMPT),
            HumanMessage(content=user_query),
        ]

        # ChatGPT 모델 호출
        response = chat_model(messages)
        return response.content

    except Exception as e:
        return f"❌ 오류 발생: {str(e)}"


# 단일 질문 테스트 함수
def test_single_query(query, show_separator=True):
    """
    단일 질문 테스트 및 결과 출력 함수

    Args:
        query (str): 테스트할 질문
        show_separator (bool): 구분선 표시 여부
    """
    print(f"📝 입력: {query}")
    result = analyze_query(query)
    print(f"📤 출력:")
    print(result)

    if show_separator:
        print("-" * 50)


print("✅ 함수 정의 완료")

✅ 함수 정의 완료


In [9]:
# 테스트용 예상 질문 10개 정의
test_queries = [
    # 기본 분리 테스트
    "일자리 복지 문화",
    # 복합 질문 (나/또는)
    "서울에 있는 일자리나 복지가 있는지 알려줘",
    # 다중 조건 분리
    "2025년에 부산에서 할 수 있는 일자리의 연령하고 전공이 궁금해",
    # 단일 질문 (분리 안함)
    "대구에서 할 수 있는 일자리의 개수는 총 몇 개야?",
    # 비교분석 (분리 안함)
    "서울과 부산의 일자리 정책 비교분석해줘",
    # 엣지 케이스 (특수문자)
    "일자리 🔍 복지문화???",
    # 불완전 문장
    "서울에서... 일자리?",
    # 복합 명령문
    "청년 창업 지원금하고 취업 준비 프로그램 찾아줘",
    # 단일 의도
    "주거 복지 정책에 대해 알려줘",
    # 긴 복합 질문
    "인천 지역 문화 예술 활동 지원 사업과 교육 프로그램 정보를 알고 싶어",
]

# 테스트 데이터 정보 출력
print("✅ 테스트 데이터 준비 완료")
print(f"📊 총 테스트 질문 개수: {len(test_queries)}개")
print("\n📋 테스트 질문 목록:")
for i, query in enumerate(test_queries, 1):
    print(f"{i:2d}. {query}")

✅ 테스트 데이터 준비 완료
📊 총 테스트 질문 개수: 10개

📋 테스트 질문 목록:
 1. 일자리 복지 문화
 2. 서울에 있는 일자리나 복지가 있는지 알려줘
 3. 2025년에 부산에서 할 수 있는 일자리의 연령하고 전공이 궁금해
 4. 대구에서 할 수 있는 일자리의 개수는 총 몇 개야?
 5. 서울과 부산의 일자리 정책 비교분석해줘
 6. 일자리 🔍 복지문화???
 7. 서울에서... 일자리?
 8. 청년 창업 지원금하고 취업 준비 프로그램 찾아줘
 9. 주거 복지 정책에 대해 알려줘
10. 인천 지역 문화 예술 활동 지원 사업과 교육 프로그램 정보를 알고 싶어


In [10]:
# 개별 테스트 실행 - 1~3번 질문
print("=== 개별 테스트 (1-3번) ===\n")

for i in range(3):
    print(f"🔍 테스트 {i+1}:")
    test_single_query(test_queries[i])

=== 개별 테스트 (1-3번) ===

🔍 테스트 1:
📝 입력: 일자리 복지 문화


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_41432\1196269613.py:20: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat_model(messages)


📤 출력:
● 일자리  
● 복지  
● 문화
--------------------------------------------------
🔍 테스트 2:
📝 입력: 서울에 있는 일자리나 복지가 있는지 알려줘
📤 출력:
● 서울에 있는 일자리 알려줘.  
● 서울에 있는 복지 알려줘.
--------------------------------------------------
🔍 테스트 3:
📝 입력: 2025년에 부산에서 할 수 있는 일자리의 연령하고 전공이 궁금해
📤 출력:
● 2025년에 부산에서 할 수 있는 일자리의 연령이 궁금해.  
● 2025년에 부산에서 할 수 있는 일자리의 전공이 궁금해.
--------------------------------------------------


In [11]:
# 개별 테스트 실행 - 4~6번 질문
print("=== 개별 테스트 (4-6번) ===\n")

for i in range(3, 6):
    print(f"🔍 테스트 {i+1}:")
    test_single_query(test_queries[i])

=== 개별 테스트 (4-6번) ===

🔍 테스트 4:
📝 입력: 대구에서 할 수 있는 일자리의 개수는 총 몇 개야?
📤 출력:
● 대구에서 할 수 있는 일자리의 개수는 총 몇 개야?
--------------------------------------------------
🔍 테스트 5:
📝 입력: 서울과 부산의 일자리 정책 비교분석해줘
📤 출력:
● 서울과 부산의 일자리 정책 비교분석해줘.
--------------------------------------------------
🔍 테스트 6:
📝 입력: 일자리 🔍 복지문화???
📤 출력:
● 일자리  
● 복지문화
--------------------------------------------------


In [12]:
# 개별 테스트 실행 - 7~10번 질문
print("=== 개별 테스트 (7-10번) ===\n")

for i in range(6, 10):
    print(f"🔍 테스트 {i+1}:")
    test_single_query(test_queries[i])

=== 개별 테스트 (7-10번) ===

🔍 테스트 7:
📝 입력: 서울에서... 일자리?
📤 출력:
● 서울에서... 일자리?
--------------------------------------------------
🔍 테스트 8:
📝 입력: 청년 창업 지원금하고 취업 준비 프로그램 찾아줘
📤 출력:
● 청년 창업 지원금 찾아줘.  
● 취업 준비 프로그램 찾아줘.
--------------------------------------------------
🔍 테스트 9:
📝 입력: 주거 복지 정책에 대해 알려줘
📤 출력:
● 주거 복지 정책에 대해 알려줘.
--------------------------------------------------
🔍 테스트 10:
📝 입력: 인천 지역 문화 예술 활동 지원 사업과 교육 프로그램 정보를 알고 싶어
📤 출력:
● 인천 지역 문화 예술 활동 지원 사업 정보 알고 싶어.  
● 인천 지역 교육 프로그램 정보 알고 싶어.
--------------------------------------------------


In [ ]:
# 전체 배치 테스트 실행
print("=== 전체 배치 테스트 ===\n")

for i, query in enumerate(test_queries, 1):
    print(f"🔍 테스트 {i}:")
    test_single_query(query)

In [13]:
# 테스트 결과 분석 함수
def analyze_test_results():
    """
    테스트 결과를 분석하여 성공률을 계산하는 함수
    """
    print("=== 테스트 결과 분석 ===\n")

    success_count = 0
    total_count = len(test_queries)
    results = []

    for i, query in enumerate(test_queries, 1):
        result = analyze_query(query)

        # 기본적인 성공 기준 체크
        # 1. ● 기호가 포함되어 있는지
        # 2. 오류 메시지가 없는지
        # 3. 빈 응답이 아닌지
        if "●" in result and "❌ 오류" not in result and result.strip():
            success_count += 1
            status = "✅ 성공"
        else:
            status = "❌ 실패"

        results.append({"query": query, "result": result, "status": status})

        print(f"{i:2d}. {query[:30]}{'...' if len(query) > 30 else ''} - {status}")

    # 전체 통계 출력
    print(f"\n📊 테스트 결과 통계:")
    print(f"   총 테스트: {total_count}개")
    print(f"   성공: {success_count}개")
    print(f"   실패: {total_count - success_count}개")
    print(f"   성공률: {(success_count/total_count)*100:.1f}%")

    return results


# 테스트 결과 분석 실행
test_results = analyze_test_results()

=== 테스트 결과 분석 ===

 1. 일자리 복지 문화 - ✅ 성공
 2. 서울에 있는 일자리나 복지가 있는지 알려줘 - ✅ 성공
 3. 2025년에 부산에서 할 수 있는 일자리의 연령하고 전... - ✅ 성공
 4. 대구에서 할 수 있는 일자리의 개수는 총 몇 개야? - ✅ 성공
 5. 서울과 부산의 일자리 정책 비교분석해줘 - ✅ 성공
 6. 일자리 🔍 복지문화??? - ✅ 성공
 7. 서울에서... 일자리? - ✅ 성공
 8. 청년 창업 지원금하고 취업 준비 프로그램 찾아줘 - ✅ 성공
 9. 주거 복지 정책에 대해 알려줘 - ✅ 성공
10. 인천 지역 문화 예술 활동 지원 사업과 교육 프로그램 ... - ✅ 성공

📊 테스트 결과 통계:
   총 테스트: 10개
   성공: 10개
   실패: 0개
   성공률: 100.0%


In [14]:
# 사용자 정의 질문 테스트
print("=== 사용자 정의 테스트 ===\n")

# 여기에 원하는 질문을 입력하여 테스트해보세요
custom_queries = [
    "대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘",
    "주거 지원 정책",
    "서울 청년 문화 활동 지원하고 부산 창업 프로그램 찾아줘",
]

for i, query in enumerate(custom_queries, 1):
    print(f"🔍 사용자 정의 테스트 {i}:")
    test_single_query(query)

=== 사용자 정의 테스트 ===

🔍 사용자 정의 테스트 1:
📝 입력: 대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘
📤 출력:
● 대학생 장학금 정보 알려줘.  
● 청년 취업 지원 프로그램 정보 알려줘.
--------------------------------------------------
🔍 사용자 정의 테스트 2:
📝 입력: 주거 지원 정책
📤 출력:
● 주거 지원 정책
--------------------------------------------------
🔍 사용자 정의 테스트 3:
📝 입력: 서울 청년 문화 활동 지원하고 부산 창업 프로그램 찾아줘
📤 출력:
● 서울 청년 문화 활동 지원해줘.  
● 부산 창업 프로그램 찾아줘.
--------------------------------------------------


In [ ]:
# 실시간 테스트 인터페이스 (선택사항)
def interactive_test():
    """
    실시간으로 질문을 입력하여 테스트할 수 있는 인터페이스
    """
    print("=== 실시간 테스트 인터페이스 ===")
    print("💡 질문을 입력하세요 (종료하려면 'quit' 또는 'exit' 입력)")
    print("💡 Jupyter에서는 Ctrl+C로 중단 가능합니다.")

    test_count = 0
    while True:
        try:
            user_input = input(f"\n[{test_count + 1}] 질문: ")

            if user_input.lower() in ["quit", "exit", "종료"]:
                print("👋 테스트를 종료합니다.")
                break

            if user_input.strip():
                test_count += 1
                print(f"📤 결과:")
                result = analyze_query(user_input)
                print(result)
                print("-" * 40)
            else:
                print("⚠️ 빈 질문입니다. 다시 입력해주세요.")

        except KeyboardInterrupt:
            print("\n👋 테스트를 중단합니다.")
            break
        except Exception as e:
            print(f"❌ 오류 발생: {e}")


# 실시간 테스트 실행 (필요시 주석 해제)
# interactive_test()

print("✅ 실시간 테스트 인터페이스 준비 완료")
print("💡 interactive_test() 함수를 호출하면 실시간 테스트를 시작할 수 있습니다.")

In [15]:
# 성능 측정 및 디버깅 도구
import time


def performance_test():
    """
    응답 시간과 토큰 사용량을 측정하는 함수
    """
    print("=== 성능 측정 테스트 ===\n")

    total_time = 0
    sample_queries = test_queries[:5]  # 처음 5개 질문으로 테스트

    for i, query in enumerate(sample_queries, 1):
        start_time = time.time()
        result = analyze_query(query)
        end_time = time.time()

        response_time = end_time - start_time
        total_time += response_time

        print(f"🔍 테스트 {i}: {query[:30]}{'...' if len(query) > 30 else ''}")
        print(f"   ⏱️ 응답시간: {response_time:.2f}초")
        print(f"   📝 응답길이: {len(result)} 글자")
        print(f"   📊 분리수: {result.count('●')}개")
        print("-" * 40)

    avg_time = total_time / len(sample_queries)
    print(f"📊 평균 응답시간: {avg_time:.2f}초")
    print(f"📊 총 소요시간: {total_time:.2f}초")


# 성능 테스트 실행
performance_test()

=== 성능 측정 테스트 ===

🔍 테스트 1: 일자리 복지 문화
   ⏱️ 응답시간: 0.79초
   📝 응답길이: 19 글자
   📊 분리수: 3개
----------------------------------------
🔍 테스트 2: 서울에 있는 일자리나 복지가 있는지 알려줘
   ⏱️ 응답시간: 1.12초
   📝 응답길이: 36 글자
   📊 분리수: 2개
----------------------------------------
🔍 테스트 3: 2025년에 부산에서 할 수 있는 일자리의 연령하고 전...
   ⏱️ 응답시간: 1.51초
   📝 응답길이: 71 글자
   📊 분리수: 2개
----------------------------------------
🔍 테스트 4: 대구에서 할 수 있는 일자리의 개수는 총 몇 개야?
   ⏱️ 응답시간: 0.71초
   📝 응답길이: 30 글자
   📊 분리수: 1개
----------------------------------------
🔍 테스트 5: 서울과 부산의 일자리 정책 비교분석해줘
   ⏱️ 응답시간: 0.83초
   📝 응답길이: 24 글자
   📊 분리수: 1개
----------------------------------------
📊 평균 응답시간: 0.99초
📊 총 소요시간: 4.97초


In [16]:
# 최종 테스트 정리 및 요약
print("=== 온통청년 질의문석 시스템 테스트 완료 ===\n")

print("🎯 테스트 완료 항목:")
print("   ✅ 라이브러리 설치 및 초기화")
print("   ✅ OpenAI API 연결")
print("   ✅ 시스템 프롬프트 설정")
print("   ✅ 기본 테스트 케이스 (10개)")
print("   ✅ 개별 테스트")
print("   ✅ 배치 테스트")
print("   ✅ 결과 분석")
print("   ✅ 사용자 정의 테스트")
print("   ✅ 성능 측정")

print("\n📋 다음 단계:")
print("   1. 실패한 테스트 케이스 분석")
print("   2. 프롬프트 개선 사항 도출")
print("   3. 추가 엣지 케이스 발견 및 테스트")
print("   4. 실제 온통청년 API와 연동 테스트")

print("\n💡 추가 기능:")
print("   - interactive_test() 함수로 실시간 테스트 가능")
print("   - 새로운 테스트 케이스 추가하여 재테스트 가능")
print("   - 성능 최적화 및 오류 패턴 분석 가능")

print("\n🔧 개선 가능한 영역:")
print("   - 복잡한 복합 질문 처리")
print("   - 문맥 이해 향상")
print("   - 도메인 특화 용어 처리")
print("   - 응답 시간 최적화")

=== 온통청년 질의문석 시스템 테스트 완료 ===

🎯 테스트 완료 항목:
   ✅ 라이브러리 설치 및 초기화
   ✅ OpenAI API 연결
   ✅ 시스템 프롬프트 설정
   ✅ 기본 테스트 케이스 (10개)
   ✅ 개별 테스트
   ✅ 배치 테스트
   ✅ 결과 분석
   ✅ 사용자 정의 테스트
   ✅ 성능 측정

📋 다음 단계:
   1. 실패한 테스트 케이스 분석
   2. 프롬프트 개선 사항 도출
   3. 추가 엣지 케이스 발견 및 테스트
   4. 실제 온통청년 API와 연동 테스트

💡 추가 기능:
   - interactive_test() 함수로 실시간 테스트 가능
   - 새로운 테스트 케이스 추가하여 재테스트 가능
   - 성능 최적화 및 오류 패턴 분석 가능

🔧 개선 가능한 영역:
   - 복잡한 복합 질문 처리
   - 문맥 이해 향상
   - 도메인 특화 용어 처리
   - 응답 시간 최적화
